In [2]:
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

C:\Users\rafae\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
model_weights_checkpoint = 'checkpoint_fc_weights.h5'
train_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb\\train'
validation_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb\\validation'
nb_train_samples = 100000
nb_validation_samples = 3024
epochs = 50
batch_size = 16

In [4]:
import keras
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import Callback

## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
history = LossHistory()

## Callback for early stopping the training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

checkpoint = keras.callbacks.ModelCheckpoint(model_weights_checkpoint, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [5]:
# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False)

In [6]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(19, activation='softmax')(x)

In [7]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [10]:
# train the model on the new data for a few epochs
#model.fit_generator(...)

In [11]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [12]:
datagen = ImageDataGenerator(rescale=1. / 255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.1,
        channel_shift_range=0.1)

generator_train = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    #class_mode=None,
    class_mode='categorical',
    shuffle=False)

generator_val = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    #class_mode=None,
    class_mode='categorical',
    shuffle=False)

Found 17528 images belonging to 19 classes.
Found 3024 images belonging to 19 classes.


In [ ]:
# train the model on the new data for a few epochs
# fine-tune the model
model.fit_generator(
    generator_train,
    #samples_per_epoch=nb_train_samples,
    steps_per_epoch = nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=generator_val,
    nb_val_samples=nb_validation_samples,
    callbacks = [checkpoint, history,early_stopping ])

C:\Users\rafae\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\rafae\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., callbacks=[<keras.ca..., validation_steps=3024, epochs=50, steps_per_epoch=6250, validation_data=<keras.pre...)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/50
